# Mise à jour de la BDD de NFT
Source: [binance.com](https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1)

In [1]:
import os
import time
import requests
import numpy as np
from utils import *
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## 1. Ouverture du marketplace

- Paramétrage du navigateur distant
- Chargement de la page d'accueil
- Activation des cookies 

In [ ]:
%%time
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
browser = webdriver.Remote("http://selenium:4444/wd/hub", options=options)
browser.get(start_url)
cookies = browser.find_element_by_xpath("//button[contains(text(),'Accept')]")
cookies.click()

## 2. Défilement des pages sur le marketplace
 - Choix du nombre de nft à récupérer
 - Chargement des pages contenant les nfts à récuperer 

In [ ]:
%%time

MAX_NFT_NB = 10#select_number()
CURRENT_PAGE_NUMBER = int(browser.current_url.split("page=")[-1].split("&")[0])
NFT_NUMBER_PER_PAGE = int(browser.current_url.split("rows=")[-1].split("&")[0])
if MAX_NFT_NB/NFT_NUMBER_PER_PAGE == MAX_NFT_NB//NFT_NUMBER_PER_PAGE:
    scroll_down = np.arange((MAX_NFT_NB//NFT_NUMBER_PER_PAGE)-1)
else:
    scroll_down = np.arange(MAX_NFT_NB//NFT_NUMBER_PER_PAGE)
for i in tqdm(scroll_down, postfix=' Scrolling down on the main page'):
    while CURRENT_PAGE_NUMBER == (browser.current_url.split("page=")[-1]).split("&")[0]:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    CURRENT_PAGE_NUMBER = (browser.current_url.split("page=")[-1]).split("&")[0]

## 3. Récupération des liens des pages détaillées
- Détection des boutons donnant accès aux infos détaillées

In [ ]:
%%time

detailed_pages = []
pbar = tqdm(total = MAX_NFT_NB, postfix=" Searching of NFT detailed pages")
BAR_LEVEL = 0
while len(detailed_pages) < MAX_NFT_NB:
  browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  detailed_pages = browser.find_elements_by_xpath('//button[contains(text(),"BSC")]')
  pbar.update(max(0,len(detailed_pages)-BAR_LEVEL))
  BAR_LEVEL = len(detailed_pages)
pbar.close()

## 4. Parcours des pages détaillés
- Récupération des données utiles sur les nft
- Nettoyage des chaînes de caractères
- Stockage des dictionnaires de données
- Fermeture du navigateur distant

In [ ]:
%%time
success = 0
failure = 0
total = len(detailed_pages)

for page in tqdm(detailed_pages, postfix=" Scraping of detailed NFT pages"):
    page.click()
    browser.switch_to.window(browser.window_handles[-1])
    res= parse_nft(browser)
    if res.status_code == 201:
        success += 1
    else:
        failure += 1
    browser.close()
    browser.switch_to.window(browser.window_handles[-1])

print(f"Success={success}/{total}; Failure={failure}/{total} - {res.status_code}")
print(f"NFT collection shape: {requests.get('http://api:8000/nfts/').text.count('_id')}")
browser.quit()

## 5. Automatisation du scraping de nfts
- Lancement du processus toutes les 5h

In [2]:
from pymongo import MongoClient
client = MongoClient('mongodb://mongodb:27017/')
database = client['local']
collection = database["nfts"]
print(collection.find_one())
collection.drop()

In [3]:
#%%writefile nft.py

from utils import *

WAIT = 2*60 # 5h = 18000s
MAX_NFT_NB = 16
NEXT_START = datetime.now()+timedelta(seconds=WAIT)
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
browser= None

while True:
    browser = open_binance(start_url, browser) # ouverture du marketplace
    scroll_down(browser, MAX_NFT_NB) # scroll down sur marketplace
    detailed_pages = find_detailed_nft_pages(browser, MAX_NFT_NB) # recherche de pages détaillées
    parse_detailed_nft_pages(browser, detailed_pages) # scraping des pages détaillées
    NEXT_START = wait_next_scraping(NEXT_START, WAIT) # attente avant prochain scraping 

2021-10-10 19:51:04: New scraping initiated
2021-10-10 19:51:04: Selenium configuration
2021-10-10 19:51:26: Binance cookies activation
2021-10-10 19:51:26: Binance page correctly opened


2021-10-10 19:51:26: Scrolling down on the main page: 0it [00:00, ?it/s]
2021-10-10 19:51:27: Searching of NFT detailed pages: 100%|██████████| 16/16 [00:00<00:00, 23.17it/s]
2021-10-10 19:51:27: Scraping of detailed NFT pages: 100%|██████████| 16/16 [00:57<00:00,  3.59s/it]


2021-10-10 19:52:25: Storage on database (Success=16/16; Failure=0/16)


2021-10-10 19:52:25: Waiting for next scraping: 100%|██████████| 39/39 [00:39<00:00,  1.00s/it]


2021-10-10 19:53:04: New scraping initiated
2021-10-10 19:53:05: Binance page correctly opened


2021-10-10 19:53:06: Scrolling down on the main page: 0it [00:00, ?it/s]
2021-10-10 19:53:06: Searching of NFT detailed pages: 100%|██████████| 16/16 [00:00<00:00, 81.87it/s]
2021-10-10 19:53:06: Scraping of detailed NFT pages: 100%|██████████| 16/16 [00:42<00:00,  2.63s/it]


2021-10-10 19:53:48: Storage on database (Success=16/16; Failure=0/16)


2021-10-10 19:53:48: Waiting for next scraping:   5%|▌         | 4/76 [00:04<01:15,  1.06s/it]


KeyboardInterrupt: 

## 6. Automatisation du scraping de cryptos
- Lancement du processus toutes les 15 minutes

In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://mongodb:27017/')
database = client['local']
collection = database["cryptos"]
collection.find_one()
collection.drop()

In [1]:
#%%writefile crypto.py

from utils import *

#WAIT = 3*60 # in seconds
WAIT = 1 # 30s
NEXT_START = datetime.now()+timedelta(seconds=WAIT)
start_url = 'https://www.binance.com/fr/markets'
browser = None

while True:
    browser = open_binance(start_url, browser, handle_cookies=False) # ouverture de la page principale des cryptos
    nb_pages = find_crypto_pages(browser) # recherche de pages additionnelles
    parse_crypto_pages(browser, nb_pages) # scraping des pages de cryptos
    NEXT_START = wait_next_scraping(NEXT_START, WAIT) # attente avant prochain scraping

2021-10-11 11:16:57: New scraping initiated
2021-10-11 11:16:57: Selenium configuration
2021-10-11 11:17:17: Binance page correctly opened
2021-10-11 11:17:17: Searching of crypto pages


2021-10-11 11:17:19: Scraping of crypto pages: 100%|██████████| 19/19 [00:18<00:00,  1.05it/s]


2021-10-11 11:17:46: Storage on database (Success=368/368; Failure=0/368)


2021-10-11 11:17:46: Waiting for next scraping: 0it [00:00, ?it/s]

2021-10-11 11:17:46: New scraping initiated


...

2021-10-11 11:18:50: Waiting for next scraping: 0it [00:00, ?it/s]

2021-10-11 11:18:50: New scraping initiated


2021-10-11 11:18:52: Binance page correctly opened
2021-10-11 11:18:52: Searching of crypto pages


2021-10-11 11:18:53: Scraping of crypto pages:  79%|███████▉  | 15/19 [00:10<00:02,  1.41it/s]


KeyboardInterrupt: 